[View in Colaboratory](https://colab.research.google.com/github/koyopro/kaggle/blob/master/recruit/recruit.ipynb)

# Recruit Restaurant Visitor Forecasting

https://www.kaggle.com/c/recruit-restaurant-visitor-forecasting  
2018/05/17

- 予約数から客数を予測する問題
- お店毎に2017/4/23~2017/5/31の数を予測する？
  - ゴールデンウィークを含むことに注意
- 日付データは2016/01/01~2017/05/31
- お店は全150店舗
- 学習データ(実際の来店客数)はair_visit_data.csv
  - 2016/01~2017/04/22までのデータ
- 学習データとテストデータはairのIDなので、先にhpgは無視してairだけでやってみるのもアリか。

In [0]:
import pandas as pd
import os
import requests
import xgboost as xgb
import numpy as np
from sklearn import preprocessing
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline 
from google.colab import files

In [2]:
open('storage.py', 'wb').write(requests.get('https://www.dropbox.com/s/xaohy7hkltic6kv/storage.py?dl=1', allow_redirects=True).content)
import storage
storage.sync_from_storage('common')

!pip install kaggle

gsutil rsync -r gs://staging.kaggle-ky.appspot.com/common .

  Running setup.py bdist_wheel for kaggle ... - \ done
  Stored in directory: /content/.cache/pip/wheels/d6/3c/92/375decc0cf0a65b281db3cca9b8e21dc27b8ebc4d1ce29afca
Successfully built kaggle


In [3]:
ls

datalab/  __pycache__/  storage.py


In [4]:
storage.sync_from_storage('recruit')

gsutil rsync -r gs://staging.kaggle-ky.appspot.com/recruit .



In [5]:
ls

air_reserve.csv     date_info.csv       sample_submission.csv
air_store_info.csv  hpg_reserve.csv     storage.py
air_visit_data.csv  hpg_store_info.csv  store_id_relation.csv
datalab/            __pycache__/


In [0]:
submission = pd.read_csv('sample_submission.csv')

In [7]:
submission.head()

,id,visitors
0,air_00a91d42b08b08d9_2017-04-23,0
1,air_00a91d42b08b08d9_2017-04-24,0
2,air_00a91d42b08b08d9_2017-04-25,0
3,air_00a91d42b08b08d9_2017-04-26,0
4,air_00a91d42b08b08d9_2017-04-27,0


In [8]:
submission.tail()

,id,visitors
32014,air_fff68b929994bfbd_2017-05-27,0
32015,air_fff68b929994bfbd_2017-05-28,0
32016,air_fff68b929994bfbd_2017-05-29,0
32017,air_fff68b929994bfbd_2017-05-30,0
32018,air_fff68b929994bfbd_2017-05-31,0


In [0]:
date_info = pd.read_csv('date_info.csv')

In [10]:
date_info.head()

,calendar_date,day_of_week,holiday_flg
0,2016-01-01,Friday,1
1,2016-01-02,Saturday,1
2,2016-01-03,Sunday,1
3,2016-01-04,Monday,0
4,2016-01-05,Tuesday,0


In [11]:
date_info.tail()

,calendar_date,day_of_week,holiday_flg
512,2017-05-27,Saturday,0
513,2017-05-28,Sunday,0
514,2017-05-29,Monday,0
515,2017-05-30,Tuesday,0
516,2017-05-31,Wednesday,0


In [12]:
air_reserve = pd.read_csv('air_reserve.csv')
air_reserve.head()

,air_store_id,visit_datetime,reserve_datetime,reserve_visitors
0,air_877f79706adbfb06,2016-01-01 19:00:00,2016-01-01 16:00:00,1
1,air_db4b38ebe7a7ceff,2016-01-01 19:00:00,2016-01-01 19:00:00,3
2,air_db4b38ebe7a7ceff,2016-01-01 19:00:00,2016-01-01 19:00:00,6
3,air_877f79706adbfb06,2016-01-01 20:00:00,2016-01-01 16:00:00,2
4,air_db80363d35f10926,2016-01-01 20:00:00,2016-01-01 01:00:00,5


In [13]:
air_reserve.tail()

,air_store_id,visit_datetime,reserve_datetime,reserve_visitors
92373,air_db4b38ebe7a7ceff,2017-05-29 18:00:00,2017-04-17 19:00:00,6
92374,air_1033310359ceeac1,2017-05-30 23:00:00,2017-04-19 17:00:00,8
92375,air_877f79706adbfb06,2017-05-31 20:00:00,2017-04-12 23:00:00,3
92376,air_900d755ebd2f7bbd,2017-05-31 20:00:00,2017-04-19 16:00:00,10
92377,air_3cad29d1a23209d2,2017-05-31 21:00:00,2017-04-22 10:00:00,3


In [0]:
hpg_reserve = pd.read_csv('hpg_reserve.csv')

In [15]:
hpg_reserve.head()

,hpg_store_id,visit_datetime,reserve_datetime,reserve_visitors
0,hpg_c63f6f42e088e50f,2016-01-01 11:00:00,2016-01-01 09:00:00,1
1,hpg_dac72789163a3f47,2016-01-01 13:00:00,2016-01-01 06:00:00,3
2,hpg_c8e24dcf51ca1eb5,2016-01-01 16:00:00,2016-01-01 14:00:00,2
3,hpg_24bb207e5fd49d4a,2016-01-01 17:00:00,2016-01-01 11:00:00,5
4,hpg_25291c542ebb3bc2,2016-01-01 17:00:00,2016-01-01 03:00:00,13


In [16]:
hpg_reserve.tail()

,hpg_store_id,visit_datetime,reserve_datetime,reserve_visitors
2000315,hpg_2b293477ea4642fb,2017-05-31 20:00:00,2017-04-19 07:00:00,2
2000316,hpg_f922aeca48ca5a29,2017-05-31 20:00:00,2017-03-19 00:00:00,3
2000317,hpg_e9151de687b93aa3,2017-05-31 21:00:00,2017-04-14 20:00:00,49
2000318,hpg_fae6c96189b4a122,2017-05-31 21:00:00,2017-04-19 17:00:00,9
2000319,hpg_0b70de808b55ad1e,2017-05-31 23:00:00,2017-04-21 13:00:00,6


In [0]:
store_id_relation = pd.read_csv('store_id_relation.csv')

In [18]:
store_id_relation.head()

,air_store_id,hpg_store_id
0,air_63b13c56b7201bd9,hpg_4bc649e72e2a239a
1,air_a24bf50c3e90d583,hpg_c34b496d0305a809
2,air_c7f78b4f3cba33ff,hpg_cd8ae0d9bbd58ff9
3,air_947eb2cae4f3e8f2,hpg_de24ea49dc25d6b8
4,air_965b2e0cf4119003,hpg_653238a84804d8e7


In [19]:
store_id_relation.shape

(150, 2)

In [0]:
air_visit_data = pd.read_csv('air_visit_data.csv')

In [21]:
air_visit_data.head()

,air_store_id,visit_date,visitors
0,air_ba937bf13d40fb24,2016-01-13,25
1,air_ba937bf13d40fb24,2016-01-14,32
2,air_ba937bf13d40fb24,2016-01-15,29
3,air_ba937bf13d40fb24,2016-01-16,22
4,air_ba937bf13d40fb24,2016-01-18,6


In [22]:
air_visit_data.tail()

,air_store_id,visit_date,visitors
252103,air_24e8414b9b07decb,2017-04-18,6
252104,air_24e8414b9b07decb,2017-04-19,6
252105,air_24e8414b9b07decb,2017-04-20,7
252106,air_24e8414b9b07decb,2017-04-21,8
252107,air_24e8414b9b07decb,2017-04-22,5


In [23]:
dates = pd.DataFrame(np.unique(air_visit_data['visit_date']))
print(dates.head())
print(dates.tail())

            0
0  2016-01-01
1  2016-01-02
2  2016-01-03
3  2016-01-04
4  2016-01-05
              0
473  2017-04-18
474  2017-04-19
475  2017-04-20
476  2017-04-21
477  2017-04-22


In [0]:
air_store_info = pd.read_csv('air_store_info.csv')

In [25]:
air_store_info.head()

,air_store_id,air_genre_name,air_area_name,latitude,longitude
0,air_0f0cdeee6c9bf3d7,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852
1,air_7cc17a324ae5c7dc,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852
2,air_fee8dcf4d619598e,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852
3,air_a17f0778617c76e2,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852
4,air_83db5aff8f50478e,Italian/French,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599


In [26]:
air_store_info.tail()

,air_store_id,air_genre_name,air_area_name,latitude,longitude
824,air_9bf595ef095572fb,International cuisine,Tōkyō-to Shibuya-ku Shibuya,35.661777,139.704051
825,air_764f71040a413d4d,Asian,Tōkyō-to Shibuya-ku Shibuya,35.661777,139.704051
826,air_10bbe8acd943d8f6,Asian,Tōkyō-to Shibuya-ku Shibuya,35.661777,139.704051
827,air_7514d90009613cd6,Karaoke/Party,Hokkaidō Sapporo-shi Minami 3 Jōnishi,43.055460,141.340956
828,air_c8fe396d6c46275d,Karaoke/Party,Hokkaidō Sapporo-shi Minami 3 Jōnishi,43.055460,141.340956


In [27]:
air_store_info.shape

(829, 5)

In [0]:
hpg_store_info = pd.read_csv('hpg_store_info.csv')

In [29]:
hpg_store_info.head()

,hpg_store_id,hpg_genre_name,hpg_area_name,latitude,longitude
0,hpg_6622b62385aec8bf,Japanese style,Tōkyō-to Setagaya-ku Taishidō,35.643675,139.668221
1,hpg_e9e068dd49c5fa00,Japanese style,Tōkyō-to Setagaya-ku Taishidō,35.643675,139.668221
2,hpg_2976f7acb4b3a3bc,Japanese style,Tōkyō-to Setagaya-ku Taishidō,35.643675,139.668221
3,hpg_e51a522e098f024c,Japanese style,Tōkyō-to Setagaya-ku Taishidō,35.643675,139.668221
4,hpg_e3d0e1519894f275,Japanese style,Tōkyō-to Setagaya-ku Taishidō,35.643675,139.668221


In [30]:
hpg_store_info.tail()

,hpg_store_id,hpg_genre_name,hpg_area_name,latitude,longitude
4685,hpg_c6ed353a48c7c93e,Sichuan food,Tōkyō-to Chiyoda-ku None,35.695780,139.768453
4686,hpg_a58d2423360b2fb4,Sichuan food,Tōkyō-to Minato-ku Roppongi,35.666137,139.733978
4687,hpg_d2021704076cdd70,Shanghai food,Tōkyō-to Chiyoda-ku None,35.695780,139.768453
4688,hpg_602472570d65e219,Spain/Mediterranean cuisine,Tōkyō-to Toshima-ku None,35.730068,139.711742
4689,hpg_11801306ce8bfb4f,Udon/Soba,Osaka Prefecture Osaka None,34.701519,135.498859


hpg_store_id多くない…？
よく見たらair_store_idも828あるわ。
そういうことか。air_storeの828が正だけど、hpg_storeにも入ってるのはそのうち150店だけで、
予測と関係ない店も4500件くらい入ってるわけね…。

とりあえずどんな特徴でやるか。  
- air_store_id
- 休日の何日前か(休日なら0)
- 前日のvisitor数
- 前回の休日のvisitor数
- 前回の平日のvisitor数

よく見たらholiday_flgは祝日だけ1なんやな

In [0]:
df = air_visit_data.copy()

In [145]:
df.head()

,air_store_id,visit_date,visitors
0,air_ba937bf13d40fb24,2016-01-13,25
1,air_ba937bf13d40fb24,2016-01-14,32
2,air_ba937bf13d40fb24,2016-01-15,29
3,air_ba937bf13d40fb24,2016-01-16,22
4,air_ba937bf13d40fb24,2016-01-18,6


In [146]:
date_info = pd.read_csv('date_info.csv')
date_info.head()

,calendar_date,day_of_week,holiday_flg
0,2016-01-01,Friday,1
1,2016-01-02,Saturday,1
2,2016-01-03,Sunday,1
3,2016-01-04,Monday,0
4,2016-01-05,Tuesday,0


In [0]:
_df = pd.DataFrame({
    'calendar_date': ['2017-06-01', '2017-06-02', '2017-06-03', '2017-06-04', '2017-06-05', '2017-06-06', '2017-06-07'],
    'day_of_week': ['Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday', 'Monday'],
    'holiday_flg': [0, 0, 0, 0, 1, 1, 0],
})
date_info = pd.concat([date_info, _df]).reset_index(drop=True)

In [0]:
saturday = date_info['day_of_week'] == 'Saturday'
sunday = date_info['day_of_week'] == 'Sunday'
date_info.loc[saturday | sunday, 'holiday_flg'] = 1

In [0]:
for i in range(7):
  key = f'{i}_days_to_holiday'
  date_info[key] = date_info['holiday_flg'][i:].reset_index(drop=True).astype('bool')
  date_info[key] = date_info[key].astype('bool')

In [150]:
print(date_info.shape)
date_info.dtypes

(524, 10)


calendar_date        object
day_of_week          object
holiday_flg           int64
0_days_to_holiday      bool
1_days_to_holiday      bool
2_days_to_holiday      bool
3_days_to_holiday      bool
4_days_to_holiday      bool
5_days_to_holiday      bool
6_days_to_holiday      bool
dtype: object

In [151]:
date_info.head(10)

,calendar_date,day_of_week,holiday_flg,0_days_to_holiday,1_days_to_holiday,2_days_to_holiday,3_days_to_holiday,4_days_to_holiday,5_days_to_holiday,6_days_to_holiday
0,2016-01-01,Friday,1,True,True,True,False,False,False,False
1,2016-01-02,Saturday,1,True,True,False,False,False,False,False
2,2016-01-03,Sunday,1,True,False,False,False,False,False,True
3,2016-01-04,Monday,0,False,False,False,False,False,True,True
4,2016-01-05,Tuesday,0,False,False,False,False,True,True,True
5,2016-01-06,Wednesday,0,False,False,False,True,True,True,False
6,2016-01-07,Thursday,0,False,False,True,True,True,False,False
7,2016-01-08,Friday,0,False,True,True,True,False,False,False
8,2016-01-09,Saturday,1,True,True,True,False,False,False,False
9,2016-01-10,Sunday,1,True,True,False,False,False,False,True


In [152]:
date_info.tail(10)

,calendar_date,day_of_week,holiday_flg,0_days_to_holiday,1_days_to_holiday,2_days_to_holiday,3_days_to_holiday,4_days_to_holiday,5_days_to_holiday,6_days_to_holiday
514,2017-05-29,Monday,0,False,False,False,False,False,False,False
515,2017-05-30,Tuesday,0,False,False,False,False,False,False,True
516,2017-05-31,Wednesday,0,False,False,False,False,False,True,True
517,2017-06-01,Tuesday,0,False,False,False,False,True,True,False
518,2017-06-02,Wednesday,0,False,False,False,True,True,False,True
519,2017-06-03,Thursday,0,False,False,True,True,False,True,True
520,2017-06-04,Friday,0,False,True,True,False,True,True,True
521,2017-06-05,Saturday,1,True,True,False,True,True,True,True
522,2017-06-06,Sunday,1,True,False,True,True,True,True,True
523,2017-06-07,Monday,0,False,True,True,True,True,True,True


In [153]:
df.head()

,air_store_id,visit_date,visitors
0,air_ba937bf13d40fb24,2016-01-13,25
1,air_ba937bf13d40fb24,2016-01-14,32
2,air_ba937bf13d40fb24,2016-01-15,29
3,air_ba937bf13d40fb24,2016-01-16,22
4,air_ba937bf13d40fb24,2016-01-18,6


In [0]:
df = pd.merge(df, date_info, how='left', left_on='visit_date', right_on='calendar_date')

In [182]:
df = df.drop(columns=['visit_date', 'calendar_date'])
df.head(10)

,air_store_id,visitors,day_of_week,holiday_flg,0_days_to_holiday,1_days_to_holiday,2_days_to_holiday,3_days_to_holiday,4_days_to_holiday,5_days_to_holiday,6_days_to_holiday
0,air_ba937bf13d40fb24,25,Wednesday,0,False,False,False,True,True,False,False
1,air_ba937bf13d40fb24,32,Thursday,0,False,False,True,True,False,False,False
2,air_ba937bf13d40fb24,29,Friday,0,False,True,True,False,False,False,False
3,air_ba937bf13d40fb24,22,Saturday,1,True,True,False,False,False,False,False
4,air_ba937bf13d40fb24,6,Monday,0,False,False,False,False,False,True,True
5,air_ba937bf13d40fb24,9,Tuesday,0,False,False,False,False,True,True,False
6,air_ba937bf13d40fb24,31,Wednesday,0,False,False,False,True,True,False,False
7,air_ba937bf13d40fb24,21,Thursday,0,False,False,True,True,False,False,False
8,air_ba937bf13d40fb24,18,Friday,0,False,True,True,False,False,False,False
9,air_ba937bf13d40fb24,26,Saturday,1,True,True,False,False,False,False,False


In [183]:
df.dtypes

air_store_id         object
visitors              int64
day_of_week          object
holiday_flg           int64
0_days_to_holiday      bool
1_days_to_holiday      bool
2_days_to_holiday      bool
3_days_to_holiday      bool
4_days_to_holiday      bool
5_days_to_holiday      bool
6_days_to_holiday      bool
dtype: object

In [0]:
df2 = pd.get_dummies(df)

In [185]:
df2.head()

,visitors,holiday_flg,0_days_to_holiday,1_days_to_holiday,2_days_to_holiday,3_days_to_holiday,4_days_to_holiday,5_days_to_holiday,6_days_to_holiday,air_store_id_air_00a91d42b08b08d9,...,air_store_id_air_fef9ccb3ba0da2f7,air_store_id_air_ffcc2d5087e1b476,air_store_id_air_fff68b929994bfbd,day_of_week_Friday,day_of_week_Monday,day_of_week_Saturday,day_of_week_Sunday,day_of_week_Thursday,day_of_week_Tuesday,day_of_week_Wednesday
0,25,0,False,False,False,True,True,False,False,0,...,0,0,0,0,0,0,0,0,0,1
1,32,0,False,False,True,True,False,False,False,0,...,0,0,0,0,0,0,0,1,0,0
2,29,0,False,True,True,False,False,False,False,0,...,0,0,0,1,0,0,0,0,0,0
3,22,1,True,True,False,False,False,False,False,0,...,0,0,0,0,0,1,0,0,0,0
4,6,0,False,False,False,False,False,True,True,0,...,0,0,0,0,1,0,0,0,0,0


In [186]:
df2.dtypes

visitors                             int64
holiday_flg                          int64
0_days_to_holiday                     bool
1_days_to_holiday                     bool
2_days_to_holiday                     bool
3_days_to_holiday                     bool
4_days_to_holiday                     bool
5_days_to_holiday                     bool
6_days_to_holiday                     bool
air_store_id_air_00a91d42b08b08d9    uint8
air_store_id_air_0164b9927d20bcc3    uint8
air_store_id_air_0241aa3964b7f861    uint8
air_store_id_air_0328696196e46f18    uint8
air_store_id_air_034a3d5b40d5b1b1    uint8
air_store_id_air_036d4f1ee7285390    uint8
air_store_id_air_0382c794b73b51ad    uint8
air_store_id_air_03963426c9312048    uint8
air_store_id_air_04341b588bde96cd    uint8
air_store_id_air_049f6d5b402a31b2    uint8
air_store_id_air_04cae7c1bc9b2a0b    uint8
air_store_id_air_0585011fa179bcce    uint8
air_store_id_air_05c325d315cc17f5    uint8
air_store_id_air_0647f17b4dc041c8    uint8
air_store_i

In [0]:
X_train = df2.drop(columns=['visitors'])

In [190]:
y_train = df.loc[:, 'visitors']
y_train.head()

0    25
1    32
2    29
3    22
4     6
Name: visitors, dtype: int64

In [191]:
print(X_train.shape)
print(y_train.shape)

(252108, 844)
(252108,)


In [169]:
model = xgb.XGBRegressor()
model.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [170]:
submission = pd.read_csv('sample_submission.csv')
submission.head()

Exception ignored in: <bound method DMatrix.__del__ of <xgboost.core.DMatrix object at 0x7fb2e7fb5ba8>>
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/xgboost/core.py", line 368, in __del__
    if self.handle is not None:
AttributeError: 'DMatrix' object has no attribute 'handle'


,id,visitors
0,air_00a91d42b08b08d9_2017-04-23,0
1,air_00a91d42b08b08d9_2017-04-24,0
2,air_00a91d42b08b08d9_2017-04-25,0
3,air_00a91d42b08b08d9_2017-04-26,0
4,air_00a91d42b08b08d9_2017-04-27,0


In [175]:
sub = submission['id'].str.split('_', expand=True)
sub.columns = ['air', 'store_id', 'visit_date']
sub['air_store_id'] = sub['air'] + '_' + sub['store_id']
sub.head()

,air,store_id,visit_date,air_store_id
0,air,00a91d42b08b08d9,2017-04-23,air_00a91d42b08b08d9
1,air,00a91d42b08b08d9,2017-04-24,air_00a91d42b08b08d9
2,air,00a91d42b08b08d9,2017-04-25,air_00a91d42b08b08d9
3,air,00a91d42b08b08d9,2017-04-26,air_00a91d42b08b08d9
4,air,00a91d42b08b08d9,2017-04-27,air_00a91d42b08b08d9


In [188]:
test = pd.merge(sub, date_info, how='left', left_on='visit_date', right_on='calendar_date')
test = test.drop(columns=['visit_date', 'calendar_date', 'air', 'store_id'])
test.head()

,air_store_id,day_of_week,holiday_flg,0_days_to_holiday,1_days_to_holiday,2_days_to_holiday,3_days_to_holiday,4_days_to_holiday,5_days_to_holiday,6_days_to_holiday
0,air_00a91d42b08b08d9,Sunday,1,True,False,False,False,False,False,True
1,air_00a91d42b08b08d9,Monday,0,False,False,False,False,False,True,True
2,air_00a91d42b08b08d9,Tuesday,0,False,False,False,False,True,True,False
3,air_00a91d42b08b08d9,Wednesday,0,False,False,False,True,True,False,False
4,air_00a91d42b08b08d9,Thursday,0,False,False,True,True,False,False,True


In [192]:
df3 = pd.get_dummies(test)
df3.head()

,holiday_flg,0_days_to_holiday,1_days_to_holiday,2_days_to_holiday,3_days_to_holiday,4_days_to_holiday,5_days_to_holiday,6_days_to_holiday,air_store_id_air_00a91d42b08b08d9,air_store_id_air_0164b9927d20bcc3,...,air_store_id_air_fef9ccb3ba0da2f7,air_store_id_air_ffcc2d5087e1b476,air_store_id_air_fff68b929994bfbd,day_of_week_Friday,day_of_week_Monday,day_of_week_Saturday,day_of_week_Sunday,day_of_week_Thursday,day_of_week_Tuesday,day_of_week_Wednesday
0,1,True,False,False,False,False,False,True,1,0,...,0,0,0,0,0,0,1,0,0,0
1,0,False,False,False,False,False,True,True,1,0,...,0,0,0,0,1,0,0,0,0,0
2,0,False,False,False,False,True,True,False,1,0,...,0,0,0,0,0,0,0,0,1,0
3,0,False,False,False,True,True,False,False,1,0,...,0,0,0,0,0,0,0,0,0,1
4,0,False,False,True,True,False,False,True,1,0,...,0,0,0,0,0,0,0,1,0,0


In [178]:
df3.dtypes

holiday_flg                  int64
0_days_to_holiday             bool
1_days_to_holiday             bool
2_days_to_holiday             bool
3_days_to_holiday             bool
4_days_to_holiday             bool
5_days_to_holiday             bool
6_days_to_holiday             bool
air_air                      uint8
store_id_00a91d42b08b08d9    uint8
store_id_0164b9927d20bcc3    uint8
store_id_0241aa3964b7f861    uint8
store_id_0328696196e46f18    uint8
store_id_034a3d5b40d5b1b1    uint8
store_id_036d4f1ee7285390    uint8
store_id_0382c794b73b51ad    uint8
store_id_03963426c9312048    uint8
store_id_04341b588bde96cd    uint8
store_id_049f6d5b402a31b2    uint8
store_id_04cae7c1bc9b2a0b    uint8
store_id_0585011fa179bcce    uint8
store_id_05c325d315cc17f5    uint8
store_id_0647f17b4dc041c8    uint8
store_id_064e203265ee5753    uint8
store_id_066f0221b8a4d533    uint8
store_id_06f95ac5c33aca10    uint8
store_id_0728814bd98f7367    uint8
store_id_0768ab3910f7967f    uint8
store_id_07b314d8305

In [181]:
predict = model.predict(df3)

ValueError: ignored